In [6]:
import pandas as pd
import numpy as np
import time
from math import inf, sin, cos, asin, acos, pi, radians

In [7]:
def make_graph(nodes_csv, links_csv):
    class Vertex:
        def __init__(self, node):
            self.id = node

            # for forward searching
            self.visited = False
            self.g_score = inf
            self.f_score = inf
            self.adjacent = {}
            self.distance = inf
            self.prev = None
            self.heuristic = None
            self.in_open_lst = False
            
            self.lat = None
            self.lon = None

            # for reversed searching
            self.visited_R = False
            self.g_score_R = inf
            self.f_score_R = inf
            self.adjacent_R = {}
            self.distance_R = inf
            self.prev_R = None


        def add_neighbor(self, neighbor, weight=0):
            self.adjacent[neighbor] = weight

        # For reversed neighbor vertex
        def add_neighbor_R(self, neighbor, weight=0):
            self.adjacent_R[neighbor] = weight

        # 대소비교를 위함
        def __lt__(self, other):
    #         self.distance_f < other.distance_f
            self.distance_R < other.distance_R


    class Graph:
        def __init__(self):
            self.vert_dict = {}

        def add_vertex(self, node):
            new_node = Vertex(node)
            self.vert_dict[node] = new_node

        def add_edges(self, frm, to, cost=0):
            if frm not in self.vert_dict:
                self.add_vertex(frm)
            if to not in self.vert_dict:
                self.add_vertex(to)
            self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)
            # to make reversed graph
            self.vert_dict[to].add_neighbor_R(self.vert_dict[frm], cost)


    # Load data
    link_data = pd.read_csv(links_csv)
    node_data = pd.read_csv(nodes_csv)

    # check the time needed for creating graph
    start = time.time()
    
    # Creat the graph
    graph = Graph()

    for i in range(len(node_data)):
        # node 생성
        graph.add_vertex(node_data['node_id'][i])
        # coordiate 부여
        graph.vert_dict[node_data['node_id'][i]].lon = node_data['xcoord'][i]
        graph.vert_dict[node_data['node_id'][i]].lat = node_data['ycoord'][i]

    # node, edge 생성(node_data에 없는 node도 여기서 생성)
    for i in range(len(link_data)):
        graph.add_edges(link_data['begin_node_id'][i], link_data['end_node_id'][i], link_data['street_length'][i])

        graph.vert_dict[link_data['begin_node_id'][i]].lon = link_data['startX'][i]
        graph.vert_dict[link_data['begin_node_id'][i]].lat = link_data['startY'][i]
        graph.vert_dict[link_data['end_node_id'][i]].lon = link_data['endX'][i]
        graph.vert_dict[link_data['end_node_id'][i]].lat = link_data['endY'][i]

    end_time = time.time() - start
    print(">>> Graph creating time:", end_time)

    return graph

In [9]:
# make_graph('./roadnetwork_data/Ulsan_nodes.csv', './roadnetwork_data/Ulsan_edges.csv')